In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
import numpy as np
# import random
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix

In [2]:
contests = pd.read_csv('data/WorkingData.csv')

In [3]:
print(contests.columns)

Index(['Unnamed: 0', 'ContestId', 'SportName', 'VariantName', 'GameSet',
       'ContestName', 'ContestStartDatetimeEST', 'ContestEndDatetimeEST',
       'ContestPayoutDatetimeEST', 'EntryFeeAmount', 'TotalPrizeAmount',
       'MaxNumberPlayers', 'MaxEntriesPerUser', 'Entries', 'DistinctUsers',
       'Contest_Group', 'NumGames', 'DraftablePlayersInSet',
       'PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition', 'is_vault'],
      dtype='object')


## We need:
Add threshold, Number of entries 4 hours out etc. 


In [4]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

In [5]:
# Get rid of cheats
features = contests.columns.drop('Entries').drop('DistinctUsers')
# Get rid of Useless 
#drop('ContestId')
features = features.drop('Unnamed: 0').drop('GameSet').drop('ContestName').drop('is_vault')
contests = contests[features].dropna()

In [6]:
contests.head()

,ContestId,SportName,VariantName,ContestStartDatetimeEST,ContestEndDatetimeEST,ContestPayoutDatetimeEST,EntryFeeAmount,TotalPrizeAmount,MaxNumberPlayers,MaxEntriesPerUser,Contest_Group,NumGames,DraftablePlayersInSet,PaidUsersInDraftGroup,TopPrize,MaxPayoutPosition
0,7963004,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,0.25,25000.0,117500,100,Headliner,15,910.0,668396,2000.0,23665
1,7963437,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,2.00,25000.0,14375,100,Booster,15,910.0,668396,20.0,1250
2,8296224,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,100.00,5000.0,55,1,SingleEntry,15,910.0,668396,1100.0,11
3,8296544,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,50.00,5000.0,111,11,FeaturedDoubleUp,15,910.0,668396,100.0,50
4,8494213,NFL,Classic,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000,5.00,300000.0,69000,200,Headliner,15,910.0,668396,25000.0,13805


### Categorizing

In [7]:
def categorizeFeature(df, name):
    return df.assign(newCol=lb_make.fit_transform(categoricalDF[name])).rename(columns={'newCol': name+'_code'})

In [8]:
categoricalFeatures = ['SportName', 'VariantName', 'Contest_Group']
categoricalDF = contests[categoricalFeatures]
for f in categoricalFeatures:
    categoricalDF = categorizeFeature(categoricalDF, f)
categoricalDF = categoricalDF.drop(categoricalFeatures, axis=1)

categoricalDF.head(10)

,SportName_code,VariantName_code,Contest_Group_code
0,10,1,7
1,10,1,2
2,10,1,13
3,10,1,5
4,10,1,7
5,12,1,7
6,10,1,7
7,10,1,7
8,10,1,7
9,14,1,7


### Scaling

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [10]:
def scaleFeature(df, name):
    return df.assign(newCol=scaler.fit_transform(df[[name]])).rename(columns={'newCol': name+'_scaled'})

In [11]:
scaleFeatures = ['EntryFeeAmount', 'TotalPrizeAmount', 'MaxNumberPlayers', 'MaxEntriesPerUser', 
                       'NumGames', 'DraftablePlayersInSet', 'PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition']
scaleDF = contests[scaleFeatures]
for f in scaleFeatures:
    scaleDF = scaleFeature(scaleDF, f)
scaleDF = scaleDF.drop(scaleFeatures, axis=1)
scaleDF.head(10)

,EntryFeeAmount_scaled,TotalPrizeAmount_scaled,MaxNumberPlayers_scaled,MaxEntriesPerUser_scaled,NumGames_scaled,DraftablePlayersInSet_scaled,PaidUsersInDraftGroup_scaled,TopPrize_scaled,MaxPayoutPosition_scaled
0,0.000005,0.0025,0.149810,9.900000e-08,0.222222,0.582524,0.891705,0.000999,0.122835
1,0.000068,0.0025,0.018326,9.900000e-08,0.222222,0.582524,0.891705,0.000009,0.006483
2,0.003568,0.0005,0.000068,0.000000e+00,0.222222,0.582524,0.891705,0.000549,0.000052
3,0.001782,0.0005,0.000139,1.000000e-08,0.222222,0.582524,0.891705,0.000049,0.000254
4,0.000175,0.0300,0.087973,1.990000e-07,0.222222,0.582524,0.891705,0.012499,0.071654
5,0.000711,0.0300,0.021896,1.990000e-07,0.000000,0.038835,0.155794,0.014999,0.018163
6,0.000318,0.0005,0.000811,5.900000e-08,0.222222,0.582524,0.891705,0.000249,0.000722
7,0.000175,0.0005,0.001464,9.900000e-08,0.222222,0.582524,0.891705,0.000249,0.001163
8,0.000246,0.0005,0.001043,7.900000e-08,0.222222,0.582524,0.891705,0.000249,0.000851
9,0.000104,0.0015,0.007329,1.990000e-07,0.079365,0.208414,0.008572,0.000499,0.006016


### Time
time data : month, day of month, day of week, hours


In [12]:
timeFeatures = ['ContestId', 'ContestStartDatetimeEST', 'ContestEndDatetimeEST', 'ContestPayoutDatetimeEST']
mapper = {'ContestStartDatetimeEST' : 'Start', 'ContestEndDatetimeEST' : 'End', 'ContestPayoutDatetimeEST': 'Payout'}
timeDF = contests[timeFeatures]
formatString = '%Y-%m-%d %H:%M:%S'

In [13]:
timeDF.head()

,ContestId,ContestStartDatetimeEST,ContestEndDatetimeEST,ContestPayoutDatetimeEST
0,7963004,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000
1,7963437,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000
2,8296224,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000
3,8296544,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000
4,8494213,2015-09-13 13:00:00.000,2015-09-14 22:20:00.000,2015-09-15 02:27:24.000


In [14]:
def mapTime(df, name, colHeader):
    miniDF = df[['ContestId', name]]
    d = {
        'ContestId': [],
        'Minute' : [],
        'Hour' : [],
        'Day' : [],
        'Month' : [],
        'Year' : [],
        'DayOfWeek' : [],
        'DayOfYear' : [],
        'WeekOfYear' : []
    }
    for index, row in miniDF.iterrows():
        time =  pd.to_datetime(row[name], format=formatString)
        d['ContestId'].append(row['ContestId'])
        d['Minute'].append(int(time.strftime('%M')))
        d['Hour'].append(int(time.strftime('%H')))
        d['Day'].append(int(time.strftime('%d')))
        d['Month'].append(int(time.strftime('%m')))
        d['Year'].append(int(time.strftime('%Y')))
        d['DayOfWeek'].append(int(time.strftime('%w')))
        d['DayOfYear'].append(int(time.strftime('%-j')))
        d['WeekOfYear'].append(int(time.strftime('%U')))
        
    renamer = {}
    for a in d:
        if a != 'ContestId':
            renamer[a] = colHeader+"_"+a

        
    newFrame = pd.DataFrame(data=d).rename(columns=renamer)  
    miniDF = pd.merge(miniDF, newFrame, on="ContestId")
#     .drop([name], axis=1)
#     print(miniDF.head())
    return miniDF
# mapTime(timeDF.head(), 'ContestStartDatetimeEST', 'Start')

In [15]:
newTimeDF = timeDF[['ContestId']]
for f in mapper:
    newTimeDF = pd.merge(newTimeDF, mapTime(timeDF, f, mapper[f]), on="ContestId").drop(columns=[f], axis=1)

### Scale Time Data

In [16]:
newTimeDF.head()

,ContestId,Start_Minute,Start_Hour,Start_Day,Start_Month,Start_Year,Start_DayOfWeek,Start_DayOfYear,Start_WeekOfYear,End_Minute,...,End_DayOfYear,End_WeekOfYear,Payout_Minute,Payout_Hour,Payout_Day,Payout_Month,Payout_Year,Payout_DayOfWeek,Payout_DayOfYear,Payout_WeekOfYear
0,7963004,0,13,13,9,2015,0,256,37,20,...,257,37,27,2,15,9,2015,2,258,37
1,7963437,0,13,13,9,2015,0,256,37,20,...,257,37,27,2,15,9,2015,2,258,37
2,8296224,0,13,13,9,2015,0,256,37,20,...,257,37,27,2,15,9,2015,2,258,37
3,8296544,0,13,13,9,2015,0,256,37,20,...,257,37,27,2,15,9,2015,2,258,37
4,8494213,0,13,13,9,2015,0,256,37,20,...,257,37,27,2,15,9,2015,2,258,37


In [17]:
scaleTimeFeatures = newTimeDF.columns.drop('ContestId')
for f in scaleTimeFeatures:
    newTimeDF = scaleFeature(newTimeDF, f)
newTimeDF = newTimeDF.drop(scaleTimeFeatures, axis=1)

Start_Minute
Start_Hour
Start_Day
Start_Month
Start_Year
Start_DayOfWeek
Start_DayOfYear
Start_WeekOfYear
End_Minute
End_Hour
End_Day
End_Month
End_Year
End_DayOfWeek
End_DayOfYear
End_WeekOfYear
Payout_Minute
Payout_Hour
Payout_Day
Payout_Month
Payout_Year
Payout_DayOfWeek
Payout_DayOfYear
Payout_WeekOfYear


In [18]:
newTimeDF.head()

,ContestId,Start_Minute_scaled,Start_Hour_scaled,Start_Day_scaled,Start_Month_scaled,Start_Year_scaled,Start_DayOfWeek_scaled,Start_DayOfYear_scaled,Start_WeekOfYear_scaled,End_Minute_scaled,...,End_DayOfYear_scaled,End_WeekOfYear_scaled,Payout_Minute_scaled,Payout_Hour_scaled,Payout_Day_scaled,Payout_Month_scaled,Payout_Year_scaled,Payout_DayOfWeek_scaled,Payout_DayOfYear_scaled,Payout_WeekOfYear_scaled
0,7963004,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113
1,7963437,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113
2,8296224,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113
3,8296544,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113
4,8494213,0.0,0.565217,0.4,0.727273,0.0,0.0,0.69863,0.698113,0.350877,...,0.70137,0.698113,0.457627,0.086957,0.466667,0.727273,0.0,0.333333,0.70411,0.698113
